## Neural Machine Translation : English to Hindi

In [ ]:
## importing necessary packages
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import unicodedata
import time
import re
import os
import io

In [ ]:
## Download dataset from http://www.cfilt.iitb.ac.in/iitb_parallel/

data_path_english = 'prunedCorpus/pruned_train.en'
data_path_hindi = 'prunedCorpus/pruned_train.hi'

In [ ]:
english = []
hindi = []

count = 0
with open(data_path_english,'r') as f:
    for line in f:
        if count==300000:       # taking only first 3,00,000 sentences
            break    
        english.append(line.strip().lower())
        count+=1  

count = 0    
with open(data_path_hindi,'r') as f:
    for line in f:
        if count==300000:       # taking only first 3,00,000 sentences
            break
        hindi.append(line)
        count+=1 

In [ ]:
## filtering corpus to contain only sentences having length upto 30 words

english_filtered = []
hindi_filtered = []

for eng_sen , hin_sen in zip(english,hindi):
    if len(eng_sen.split())<=30 and len(hin_sen.split()) <= 30:
        english_filtered.append(eng_sen)
        hindi_filtered.append(hin_sen)

english = english_filtered 
hindi = hindi_filtered


In [ ]:
print(len(english))
print(len(hindi))

In [ ]:
def preprocess_text(corpus):
    for i in range(len(corpus)):   
        corpus[i] = re.sub(r"([?.!,।])", r" \1 ", corpus[i])
        corpus[i] = re.sub(r'[" "]+', " ", corpus[i])
        corpus[i] = re.sub(r"[^a-zA-Zऀ-ॿ?.!,¿।]+", " ", corpus[i])
        corpus[i] = corpus[i].strip()
        corpus[i] = '<start> ' + corpus[i] + ' <end>'
    return corpus    

def prepare_dataset(corpus):
    corpus = corpus.copy()
    
    corpus = preprocess_text(corpus)
    
    # tokenizing sentences
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='',lower=False)
    tokenizer.fit_on_texts(corpus)
    sequences = tokenizer.texts_to_sequences(corpus)
    
    #padding_sentences
    pad_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences,padding='post')
    
    return corpus, pad_sequences, tokenizer

In [ ]:
#creating datasets

english_corpus,english_tokenized, english_tokenizer = prepare_dataset(english) ## input
hindi_corpus,hindi_tokenized, hindi_tokenizer = prepare_dataset(hindi) ## target

In [ ]:
# train dev test split
# X : english sentences
# y : hindi sentences

train_X , test_X , train_y , test_y = train_test_split(english_tokenized,hindi_tokenized,test_size=0.1)
#train_X , val_X , train_y, val_y = train_test_split(train_X,train_y,test_size=0.2)

print(f'train set size: {len(train_X)}')
print(f'test set size: {len(test_X)}')
#print(f'validation set size: {len(val_X)}')

In [ ]:
## creating tf.data dataset

BUFFER_SIZE = len(train_X)
BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((train_X, train_y)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

max_len_hindi = next(iter(dataset))[1][0].shape[0]
max_len_english = next(iter(dataset))[0][0].shape[0]

In [ ]:
#' '.join([hindi_tokenizer.index_word.get(ind) for ind in list(next(iter(dataset))[1][0].numpy()) if english_tokenizer.index_word.get(ind) != None])

In [ ]:
steps_per_epoch = len(train_X)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_X_len = len(english_tokenizer.word_index)+1
vocab_y_len = len(hindi_tokenizer.word_index)+1

In [ ]:
## Encoder Model

class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, encoder_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.encoder_units = encoder_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.encoder_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

    def call(self, x, hidden = None):
        if hidden==None:
            hidden=tf.zeros((self.batch_size, self.encoder_units))
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

In [ ]:
encoder = Encoder(vocab_X_len,embedding_dim,units,BATCH_SIZE)
#encoder_outputs,encoder_state = encoder(next(iter(dataset))[0].numpy())

In [ ]:
#encoder_outputs.shape, encoder_state.shape

In [ ]:
## Attention Mechanism

class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))
        attention_weights = tf.nn.softmax(score, axis=1)

        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [ ]:
attention = BahdanauAttention(16)
#ctx_vector, attention_weights = attention(encoder_state,encoder_outputs)

In [ ]:
#ctx_vector.shape, attention_weights.shape

In [ ]:
## Decoder

class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, decoder_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.decoder_units = decoder_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.decoder_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.decoder_units)

    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        x = self.fc(output)

        return x, state, attention_weights

In [ ]:
decoder = Decoder(vocab_y_len,embedding_dim,units,BATCH_SIZE)

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [ ]:
#! mkdir 'training_checkpoints'
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
@tf.function
def train_step(X, y):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, dec_hidden = encoder(X)

        dec_input = tf.expand_dims([english_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

        for t in range(1, y.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(y[:, t], predictions)
            
            dec_input = tf.expand_dims(y[:, t], 1)

        batch_loss = (loss / int(y.shape[1]))

        variables = encoder.trainable_variables + decoder.trainable_variables

        gradients = tape.gradient(loss, variables)

        optimizer.apply_gradients(zip(gradients, variables))

        return batch_loss

In [ ]:
EPOCHS = 1

for epoch in range(EPOCHS):
    start = time.time()

    total_loss = 0

    for (batch, (X, y)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(X, y)
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,batch,batch_loss.numpy()))

    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
def evaluate(sentence):
    
    sentence = preprocess_text(sentence)
    
    inputs = [english_tokenizer.word_index.get(i) for i in sentence[0].strip().lower().split(' ') if english_tokenizer.word_index.get(i) != None]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_len_english, padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs,hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([english_tokenizer.word_index['<start>']], 0)

    for t in range(max_len_hindi):
        predictions, dec_hidden, attention_weights = decoder(dec_input,dec_hidden,enc_out)

        attention_weights = tf.reshape(attention_weights, (-1, ))

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += hindi_tokenizer.index_word[predicted_id] + ' '

        if hindi_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence 

        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence 

In [ ]:
def translate(sentence):
    result, sentence = evaluate(sentence)
    
    print('Input: %s' % (sentence[0]))
    print('Predicted translation: {}'.format(result))

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
translate(['wow.'])